In [ ]:
# Install the library if you are running this in Colab
!pip install py-adtools


# adtools.sandbox 教程：安全代码执行

`adtools.sandbox` 提供了一个安全的环境来执行不可信的代码（例如由 LLM 生成的代码）。它通过将执行隔离在单独的进程中来实现这一点，从而允许我们进行超时管理、资源保护和输出重定向。

## 为什么需要沙盒？

当我们在程序中自动执行 LLM 生成的代码时，可能会遇到以下风险：
1.  **无限循环**：生成的代码可能包含死循环，导致主程序卡死。
2.  **有害操作**：代码可能尝试删除文件或访问网络。
3.  **输出干扰**：代码中的 `print` 语句可能会扰乱主程序的日志。

`SandboxExecutor` 可以解决这些问题。

## 基础用法：SandboxExecutor

你可以将任何类或对象包装在 `SandboxExecutor` 中，以便在单独的进程中执行其方法。


In [ ]:
import time
from typing import Any
from adtools.sandbox.sandbox_executor import SandboxExecutor

# 定义一个简单的执行器类
class CodeRunner:
    def run_code(self, code_str: str) -> Any:
        local_scope = {}
        # 警告：exec 是不安全的，但在沙盒进程中执行可以降低风险
        exec(code_str, local_scope)
        if "main" in local_scope:
            return local_scope["main"]()
        return None

# 示例代码：一个正常的函数
safe_code = """
def main():
    return sum([i for i in range(100)])
"""

# 示例代码：一个包含无限循环的函数
infinite_loop_code = """
def main():
    while True:
        pass
"""

if __name__ == "__main__":
    # 初始化 SandboxExecutor
    # debug_mode=True 会打印更多调试信息
    runner = CodeRunner()
    sandbox = SandboxExecutor(runner, debug_mode=True)

    print("正在执行安全代码...")
    # 安全执行 run_code 方法
    result = sandbox.secure_execute(
        "run_code",
        method_args=(safe_code,),
        timeout_seconds=5
    )
    print(f"结果: {result['result']}, 耗时: {result['evaluate_time']:.4f}s")

    print("正在执行无限循环代码（设置了 2 秒超时）...")
    # 执行包含死循环的代码
    result = sandbox.secure_execute(
        "run_code",
        method_args=(infinite_loop_code,),
        timeout_seconds=2
    )
    
    if result['result'] is None:
        print(f"执行失败或超时。错误信息: {result['error_msg']}")
    else:
        print(f"结果: {result['result']}")


## 输出重定向

默认情况下，子进程的输出（stdout/stderr）可能会显示在控制台上。你可以使用 `redirect_to_devnull=True` 将其静音。


In [ ]:
noisy_code = """
def main():
    print("这是一条不需要显示的日志消息")
    print("这也是一条")
    return "Done"
"""

print("执行嘈杂代码（重定向输出到 /dev/null）...")
result = sandbox.secure_execute(
    "run_code",
    method_args=(noisy_code,),
    timeout_seconds=5,
    redirect_to_devnull=True
)
print(f"结果: {result['result']}")


## Ray 沙盒执行器

`adtools` 还提供了基于 Ray 的沙盒实现 `SandboxExecutorRay`。它适用于分布式执行或需要更强隔离的场景。

要使用它，你需要安装 Ray (`pip install ray`)。

```python
from adtools.sandbox.sandbox_executor_ray import SandboxExecutorRay

# 初始化 Ray 沙盒
# init_ray=True 会自动初始化 Ray 集群（如果是单机则为本地集群）
sandbox_ray = SandboxExecutorRay(runner, init_ray=True)

result = sandbox_ray.secure_execute(
    "run_code",
    method_args=(safe_code,),
    timeout_seconds=5
)
```

Ray 沙盒的一个主要优势是支持**零拷贝**的大对象返回（例如大型 NumPy 数组或 PyTorch 张量），这在机器学习任务中非常有用。
